<a href="https://colab.research.google.com/github/ThuaneFranca/Analisador-Lexico/blob/master/Sintatico_e_Semantico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisador Lexico, Sintatico e Semantico
# Autora: Thuane França

In [0]:
#@title Módulo1 - Analisador Léxico (Rodar antes) { display-mode: "form" }

import re, sys
from collections import namedtuple 


Palavra = namedtuple('Palavra','simbolo tipo')


def abrir_programa(arquivo):
    # ABRE UM ARQUIVO E TRANSFORMA NUMA STRING
    with open(arquivo,'r') as pr:
        programa = pr.read()
        return programa



class Lexico:
    def __init__(self, arquivo):
        # INICIALIZA TODAS AS VARIAVEIS DO LEXICO

        self.fila_de_tokens = arquivo
        self.posicao_atual = 0
        self.tamanho_da_fila = len(self.fila_de_tokens)

        self.palavras = []

        self.inicio_palavra = 0
        self.final_palavra = 0

        

    def ler_caractere(self, multiplos=False):
      
        # LÊ O CARACTER NA POSIÇÃO ATUAL
        if self.posicao_atual < self.tamanho_da_fila:
            if multiplos:
                return self.fila_de_tokens[self.posicao_atual:]
            else:
                return self.fila_de_tokens[self.posicao_atual]
        else: 
            return False
    
    def proximo_caractere(self):
        # INCREMENTA A POSIÇÃO ATUAL, SE EXISTIR MAIS CARACTERES
        if self.posicao_atual < self.tamanho_da_fila:
            self.posicao_atual += 1
            self.final_palavra += 1
        else:
            return False
    
    def cadastrar_palavra(self,tipo):
        # SALVA A PALAVRA E O TIPO DO SIMBOLO NA LISTA DE PALAVRAS
        # RECEBE O TEXTO DO TIPO DO SIMBOLO: ex cadastrar_palavra('identificador')
        #print(f"{self.inicio_palavra} -  {self.final_palavra}")
        nova_palavra = Palavra(simbolo=self.fila_de_tokens[self.inicio_palavra:self.final_palavra], tipo=tipo)
        self.palavras.append(nova_palavra)

    def proximo_caractere_nao_vazio(self):
        # INCRIMENTA A POSIÇÃO ATUAL ATÉ ACHAR O PRÓXIMO CARACTERE NÃO VAZIO
        # enquanto os caracteres da fila são espaços, vai incrementando a posição
        while self.posicao_atual < self.tamanho_da_fila:
            if re.match(r'[\t\n\r\f\040]', self.fila_de_tokens[self.posicao_atual]):
                self.posicao_atual += 1
            else:
                # atualiza a posição do inicio e final de palavra
                self.inicio_palavra = self.posicao_atual
                self.final_palavra = self.posicao_atual
                #RE
                return self.fila_de_tokens[self.posicao_atual]

    def start(self, debug=False):
        #INICIA A EXECUÇÃO DO ANALISADOR
        #enquanto existir caractere, segue em frente rodando
        while self.posicao_atual < self.tamanho_da_fila:
            caractere = self.proximo_caractere_nao_vazio()

            if not caractere: break
            # utiliza regex (expressão regular) pra verificar que tipo de simbolo é,
            # e executa uma função de verificação de acordo
            elif re.match(r'[a-zA-Z_]',caractere): self.identificador()
            elif re.match(r'[0-9]',caractere): self.numero()
            elif re.match(r'[\{]',caractere): self.comentarios()
            elif re.match(r'[\.\;\(\),:+-/\*=<>]',caractere): self.especial()
            elif re.match(r'[\}]',caractere): self.comentario_quebrado()
 
            else:
                # se nada for encontrado, incrementa a posiçãso
                self.inicio_palavra += 1
                self.posicao_atual += 1

        #NO FINAL IMPRIME E RETORA A LISTA DE SIMBOLOS(TOKENS)
        if debug:
          for x in self.palavras: print(f"Simbolo: {x[0]}   - Tipo: {x[1]}")
        return self.palavras

    def identificador(self):
        # ESSA FUNÇÃO VERIFICA SE O SIMBOLO É UM IDENTIFICADOR

        # ENQUANTO TIVER CARACTERE ALFANUMERICO PRA LER, VAI LENDO E INCREMENTANDO
        # O A POSIÇÃO DOS CARACTERES, E POR FIM, SALVA A PALAVRA
        while True:
            caractere = self.ler_caractere()
            if not caractere: break
            if re.match('[a-zA-Z0-9_]', caractere): 
                self.proximo_caractere()
            else: 
                break

        # VERIFICAR SE A PALAVRA É RESERVADA
        PALAVRAS_RESERVADAS = [ 'program', 'var', 'integer', 'real', 'boolean', 
            'procedure', 'begin', 'end', 'if', 'then', 'else', 
            'while', 'do', 'not' ]

        palavra_temporaria = self.fila_de_tokens[self.inicio_palavra: self.final_palavra]
        # SE FOR RESERVADA, REGISTRA DE ACORDO COM O TIPO DO SIMBOLO RESERVADO
        if palavra_temporaria in PALAVRAS_RESERVADAS:
            self.cadastrar_palavra('reservada')
        elif palavra_temporaria == 'and':
            self.cadastrar_palavra('multiplicacao')
        elif palavra_temporaria == 'or':
            self.cadastrar_palavra('adicao')
        elif palavra_temporaria in ['true','false']:
            self.cadastrar_palavra('boolean')
        else:
            self.cadastrar_palavra('identificador')


    def numero(self):
        #VERIFICA SE É NUMERO
        # REAL SE É NUMEROS SEGUIDOS DE UM PONTO EX: 1234.56
        # INTEIRO SE SÃO APENAS NUMEROS
        # INVALIDOS SE RECEBEM ALGUMA LETRA NO FINAL
        invalido, real = False, False
        potencia = False
        
        while True:
            caractere = self.ler_caractere()
            if not caractere: 
                break
            if re.match('[0-9]',caractere): 
                self.proximo_caractere()
            elif re.match('[.]',caractere) and not real:
                real = True
                self.proximo_caractere()
            elif re.match('[.]',caractere) and real:
                invalido = True
                self.proximo_caractere()
            elif re.match('[eE]',caractere):
                pot = self.ler_caractere(multiplos=True)
                if len(pot) > 2:
                  if pot[1] in ['+','-']:
                    potencia = True
                    self.proximo_caractere()
                    self.proximo_caractere()
                  else:
                    break
                  
            else: 
                break

        if potencia: 
            self.cadastrar_palavra('potencia')
        elif real: 
            self.cadastrar_palavra('real')
        else: 
            self.cadastrar_palavra('integer')



    def especial(self):
        #print(self.palavras)

        # Simbolos Compostos de 2 caracteres
        # CARACTERES = LISTA DE TODOS OS CARACTERES SEGUINTES
        caracteres = self.ler_caractere(multiplos=True)

        if len(caracteres) > 1:
            # SE TEM MAIS DE UM CARACTERE PELA FRENTE
            # LÊ E SALVA OS DOIS
            char_1, char_2 = caracteres[0], caracteres[1]
        else:
            # CASO CONTRÁRIO, SÓ SALVA O PRIMEIRO
            char_1, char_2 = self.ler_caractere(), False

        # VERIFICAR SE É COMENTARIO DE LINHA
        if char_1 is '/' and char_2 is '/':
            self.comentario_de_linha()
        
        #SIMBOLOS DE DOIS CARACTERES
        # CASO SEJA ACEITO, PULA O PRÓXIMO CARACTERE DUAS VEZES
        elif char_1 is ':' and char_2 is '=':
            self.proximo_caractere()
            self.proximo_caractere()
            self.cadastrar_palavra('atribuicao')

        elif (char_1 is '<' and char_2 in ['=','>']) or \
        (char_1 is '>' and char_2 is '='):        
            self.proximo_caractere()
            self.proximo_caractere()
            self.cadastrar_palavra('relacao')

        elif (char_1 is '*' and char_2 is '*'):
            self.proximo_caractere()
            self.proximo_caractere()
            self.cadastrar_palavra('expoente')

        # caso não sejam dois caracteres
        # verifica e cadastra a palavra
    
        elif char_1 in ['.',';','(',')',':',',']: 
            self.proximo_caractere()
            self.cadastrar_palavra('delimitador')
        
        elif char_1 in ['=','<','>']:
            self.proximo_caractere()
            self.cadastrar_palavra('relacao')

        elif char_1 in ['+','-']: 
            self.proximo_caractere()
            self.cadastrar_palavra('adicao')
        elif char_1 in ['*','/']: 
            self.proximo_caractere()
            self.cadastrar_palavra('multiplicacao')
    

    def comentarios(self):
        # COMENTARIO: IGNORA TODAS AS PALAVRAS SEGUINTES ATÉ
        # ACHAR O SIMBOLO DE FECHAR COMENTÁRIO "}"

        while True:
            if self.posicao_atual > self.tamanho_da_fila:
                print("ERRO: COMENTARIO NÃO-FECHADO NO CÓDIGO")
                return False
            
            caractere = self.ler_caractere()

            if not caractere:
                print("ERRO: COMENTARIO NÃO-FECHADO NO CÓDIGO")
                return False
                        
            elif re.match(r'[\}]', caractere): 
                self.proximo_caractere()
                break
            elif re.match(r'[\n]',caractere): 
                self.proximo_caractere()
            else: 
                self.proximo_caractere()
  
    def comentario_de_linha(self):
        # COMENTARIO DE LINHA: IGNORA TUDO ATÉ O FINAL DA LINHA
        while True:
            caractere = self.ler_caractere()
            #print(caractere)
            if not caractere:
                return False
        
            elif re.match(r'[\n]',caractere): 
                self.proximo_caractere()
                break    
            else: 
                self.proximo_caractere()
    def comentario_quebrado(self):
      print("ERRO: COMENTARIO FECHADO SEM ABRIR")
      self.inicio_palavra += 1
      self.posicao_atual += 1





In [0]:
#@title Módulo2 - Analisador Sintático + Semantico (Rodar antes) { display-mode: "form" }


#DECORADOR 
def logar(func):
  def wrapper(self, *args, **kwargs):
    if self.debug:
      #print(f'ENTRANDO: {func.__name__} com {self.lista_de_palavras[self.posicao_atual]}')
      print(f'ESCOPO ATUAL: {self.pilha_de_escopo}')
    original_result = func(self,*args, **kwargs)
    if self.debug: 
      print(f'ESCOPO ATUAL: {self.pilha_de_escopo}')
      print(f'SAINDO {func.__name__}')
    return original_result
  return wrapper





class Sintatico(object):
  def __init__(self, lista_de_palavras, debug=False):
    self.lista_de_palavras = lista_de_palavras
    self.posicao_atual = 0
    self.tamanho_da_lista = len(self.lista_de_palavras)
    self.debug = debug
    self.pilha_de_escopo = []
    self.pilha_de_avaliacao = []




  # FUNÇÕES DE TRAVESSIA de codigo, tipo, simbolo, pala reserv
  def ler_simbolo(self):
    if self.posicao_atual < self.tamanho_da_lista:
      return self.lista_de_palavras[self.posicao_atual].simbolo
    else:
      return False

  def ler_tipo(self):
    if self.posicao_atual < self.tamanho_da_lista:
      return self.lista_de_palavras[self.posicao_atual].tipo
    else:
      return False

  def ler_palavra(self):
    if self.posicao_atual < self.tamanho_da_lista:
      return self.lista_de_palavras[self.posicao_atual]
    else:
      return False

  def proximo(self):
    self.posicao_atual += 1

  def erro(self, msg):
    print(f"ERRO: {msg}")
    print(self.lista_de_palavras[self.posicao_atual])
    raise





  # FUNÇÕES DE ESCOPO - SEMANTICO 

  def adicionar_escopo(self):
    self.pilha_de_escopo.append('$')
    #print("novo escopo")

  def remover_escopo(self):
    #reverte, remove o escopo e reverte de volta
    self.pilha_de_escopo.reverse()
    indice = self.pilha_de_escopo.index('$')
    self.pilha_de_escopo = self.pilha_de_escopo[indice:]
    self.pilha_de_escopo.reverse()
    #print("saiu do escopo")


  def adicionar_declaracao(self, palavra):
    escopo_atual = self.pilha_de_escopo[::-1]
    indice = escopo_atual.index('$')
    escopo_atual = escopo_atual[:indice]

    if palavra in escopo_atual:
      print(f'ERRO: Variavel {palavra.simbolo} já declarada nesse escopo')
      return False
    else:
      self.pilha_de_escopo.append(palavra)
      return True

  def verificar_declaracao(self, palavra):
    if palavra in self.pilha_de_escopo:
        return True
    else:
      print(f'ERRO: variavel {palavra.simbolo} não declarada')
      return False



  # VERIFICACAO DE TIPOS

  def iniciar_avaliacao(self):
    self.pilha_de_avaliacao = []
  
  def finalizar_avaliacao(self):
    try:
      x = pilha_de_avaliacao.pop()        
      while pilha_de_avaliacao:
        y = pilha_de_avaliacao.pop()
        if (x in ['integer','real'] and y == 'boolean') or (x == 'boolean' and y in ['integer','real']):
          print(f'ERRO: Proibido operação booleana com tipos numericos')
        elif (x == 'relacao' and y == 'boolean') or (x == 'boolean' and y == 'relacao'):
          print(f'ERRO: Proibido operação relacional com tipos booleanos')
    except:
      pass




  # FUNÇÕES

  #MÁQUINA DE ESTADOS 
  def start(self):
    try:
      self.programa()
    except:
      pass
  
  @logar
  def programa(self):
    """
    programa →
      program id ;
      declarações_variáveis
      declarações_de_subprogramas
      comando_composto
    """
    #adiciono o $ na minha pilha
    self.adicionar_escopo()
    #PROGRAM ID:
    if self.ler_simbolo() == 'program': 
      self.proximo()
    else: self.erro("PROGRAMA: faltando palavra reservada 'program'")

    if self.ler_tipo() == 'identificador': 
      self.adicionar_declaracao(self.ler_palavra())
      self.proximo()
    else: self.erro("PROGRAMA: faltando identificador") 
    
    if self.ler_simbolo() == ';': 
      self.proximo()
    else: self.erro("PROGRAMA: faltando ponto-e-virgula")

    # DECLARAÇÕES
    self.declaracoes_variaveis()
    self.declaracoes_de_subprogramas()
    self.comando_composto()

    # PONTO FINAL
    if self.ler_simbolo() == '.': 
      self.proximo()
    else: self.erro("PROGRAMA: faltando ponto final")

    # VERIFICAR SE PROGRAMA TERMINOU
    if not self.ler_simbolo():
      print("Análise Sintática e Semantica Concluida.")
    else: self.erro("PROGRAMA: Código depois do ponto final.")
 
  @logar
  def declaracoes_variaveis(self):
    """
    declarações_variáveis →
      var lista_declarações_variáveis
      | ε
    """
    if self.ler_simbolo() == 'var':
      self.proximo()
      self.lista_declaracoes_variaveis()

  @logar
  def lista_declaracoes_variaveis(self):
    """
    lista_declarações_variáveis →
      lista_declarações_variáveis lista_de_identificadores : tipo ;
      | lista_de_identificadores : tipo ;
    """

    self.lista_de_identificadores()
    # : tipo ;

    if self.ler_simbolo() == ':':
      self.proximo() 
      self.tipo()
    else: self.erro("LISTA DE IDENTIFICADORES: faltando ':'")
    
    if self.ler_simbolo() == ';': 
      self.proximo()
    else: self.erro("LISTA DE IDENTIFICADORES: faltando ';'")

    if self.ler_tipo() == 'identificador':
      self.lista_declaracoes_variaveis()

  @logar
  def lista_de_identificadores(self):
    """
    lista_de_identificadores →
      id
      | lista_de_identificadores , id
    """
    if self.ler_tipo() == 'identificador': 
      self.adicionar_declaracao(self.ler_palavra())
      self.proximo()
    else: self.erro("LISTA DE IDENTIFICADORES: faltando identificador")

    if self.ler_simbolo() == ',':
      self.proximo()
      self.lista_de_identificadores()

  @logar
  def tipo(self):
    """
    tipo →
      integer
      | real
      | boolean
    """

    MEUS_TIPOS = ['integer', 'real', 'boolean']

    if self.ler_simbolo() in MEUS_TIPOS: 
      self.proximo()
    else: self.erro("Faltando Declaração de TIPO")
  
  @logar
  def declaracoes_de_subprogramas(self):
    """
    declarações_de_subprogramas →
      declarações_de_subprogramas declaração_de_subprograma ;
      | ε
    """
    if self.ler_simbolo() == 'procedure':
      self.declaracao_de_subprograma()
      if self.ler_simbolo() == ';':
        self.proximo()
        self.declaracoes_de_subprogramas()
      else: self.erro("DECLARACAO DE SUBPROGRAMA: FALTANDO PONTO-E-VIRGULA")
  
  @logar
  def declaracao_de_subprograma(self):
    """
    declaração_de_subprograma →
      procedure id argumentos ;
      declarações_variáveis
      declarações_de_subprogramas
      comando_composto
    """
    if self.ler_simbolo() == 'procedure': 
      self.proximo()
    else: self.erro("DECLARAÇÃO DE SUBPROGRAMA: faltando palavra-chave procedure")

    if self.ler_tipo() == 'identificador':
      self.adicionar_declaracao(self.ler_palavra())
      self.proximo()
      self.adicionar_escopo() #começa programa, adiciona escopo
    else: self.erro("DECLARAÇÃO DE SUBPROGRAMA: faltando identificador")

    self.argumentos()

    if self.ler_simbolo() == ';':
      self.proximo()
    else: self.erro( "DECLARAÇÃO DE SUBPROGRAMA: faltando ponto-e-virgula")

    self.declaracoes_variaveis()
    self.declaracoes_de_subprogramas()
    self.comando_composto()
    self.remover_escopo() # termina programa, remove escopo

  @logar
  def argumentos(self):
    """
    argumentos →
      (lista_de_parametros)
      | ε
    """
    if self.ler_simbolo() == '(':
      self.proximo()
      self.lista_de_parametros()

      if self.ler_simbolo() == ')':
        self.proximo()
      else: self.erro( 'ARGUMENTOS: faltando simbolo ")"')

  @logar
  def lista_de_parametros(self):
    """
    lista_de_parametros →
      lista_de_identificadores : tipo
      | lista_de_parametros ; lista_de_identificadores : tipo
    """
    self.lista_de_identificadores()

    if self.ler_simbolo() == ':':
      self.proximo()
      self.tipo()
    else: 
      self.erro("PARAMETROS: faltando dois-pontos")

    if self.ler_simbolo() == ';':
      self.proximo()
      self.lista_de_parametros()
    
  @logar
  def comando_composto(self):
    """
    comando_composto →
      begin
      comandos_opcionais
      end
    """
    if self.ler_simbolo() == 'begin': 
      self.proximo()
    else: self.erro( "COMANDO COMPOSTO: faltando BEGIN")
    
    self.comandos_opcionais()

    if self.ler_simbolo() == 'end': 
        self.proximo()
    else: self.erro( "COMANDO COMPOSTO: faltando END")

  @logar
  def comandos_opcionais(self):
    """
    comandos_opcionais →
      lista_de_comandos
      | ε
    """
    if self.ler_simbolo() != 'end':
      self.lista_de_comandos()

  @logar
  def lista_de_comandos(self):
    """
    lista_de_comandos →
      comando
      | lista_de_comandos ; comando
    """
    self.comando()

    if self.ler_simbolo() == ';':
      self.proximo()
      self.lista_de_comandos()

  @logar
  def comando(self):
    """
    comando →
      variável := expressão
      | ativação_de_procedimento
      | comando_composto
      | if expressão then comando parte_else
      | while expressão do comando
    """

    # IF EXPRESSÃO THEN COMANDO PARTE_SELSE
    if self.ler_simbolo() == "if":
      self.proximo()
      self.expressao()
      
      if self.ler_simbolo() == "then":
        self.proximo()
      else: self.erro("COMANDO IF: FALTANDO THEN")
      
      self.comando()
      self.parte_else()

    # WHILE EXPRESSÃO DO COMANDO
    elif self.ler_simbolo() == "while":
      self.proximo()
      self.expressao()
      
      if self.ler_simbolo() == 'do':
        self.proximo()
      else: self.erro("COMANDO WHILE: FALTANDO DO")

      self.comando()

    # COMANDO COMPOSTO
    elif self.ler_simbolo() == 'begin':
      self.comando_composto()

    # VARIAVEL := EXPRESSÃO E ATIVAÇÃO DE PROCEDIMENTO

    elif self.ler_tipo() == 'identificador':
      self.verificar_declaracao(self.ler_palavra())
      self.proximo()
      # VARIAVEL
      if self.ler_tipo() == 'atribuicao':
        self.proximo()
        self.iniciar_avaliacao()
        self.expressao()
        self.finalizar_avaliacao()

      # ATIVACAO DE PROCEDIMENTO
      elif self.ler_simbolo() == '(':
        self.proximo()
        self.lista_de_expressoes()
        if self.ler_simbolo() == ')':
          self.proximo()
        else: 
          self.erro( 'ATIVAÇÃO PROCEDIMENTO - FALTANDO ")" ')

      else:
        # caso contrário, deu certo e é ativação de procedimento
        pass

  @logar
  def parte_else(self):
    """
    parte_else →
      else comando
      | ε
    """
    if self.ler_simbolo() == "else":
      self.proximo()
      self.comando()
      self.parte_else()

  @logar
  def variavel(self):
    """
    id
    """
    #função inclusa no comando
    pass

  @logar
  def ativacao_de_procedimento(self):
    """
    ativação_de_procedimento →
      id
      | id (lista_de_expressões)
    """
    # essa função está inclusa no COMANDO. 
    pass


  @logar
  def lista_de_expressoes(self):
    """
    lista_de_expressões →
      expressão
      | lista_de_expressões , expressão
    """

    self.expressao()

    if self.ler_simbolo() == ',': 
      self.proximo()
      self.lista_de_expressoes()

  @logar
  def expressao(self):
    """
    expressão →
      expressão_simples
      | expressão_simples op_relacional expressão_simples
    """

    self.expressao_simples()

    if self.ler_tipo() == 'relacao': 
      self._op_relacional()
      self._expressao()

    

  @logar
  def expressao_simples(self):
    """
    expressão_simples →
      termo
      | sinal termo
      | expressão_simples op_aditivo termo
    """
    if self.ler_simbolo() in ['+','-']:
      self.sinal()
      self.termo()
    else: 
      self.termo()

    # enquanto aparecer adição, loopar em busca de termos    
    if self.ler_tipo() == 'adicao':
      self.op_aditivo()
      self.expressao_simples()


  @logar
  def termo(self):
    """
    termo →
      fator
      | termo op_multiplicativo fator
    """

    self.fator()

    if self.ler_tipo() == 'multiplicacao': 
      self.op_multiplicativo()
      self.termo()
   
   
   
  @logar
  def fator(self): 
    """
    fator →
      id
      | id(lista_de_expressões)
      | num_int
      | num_real
      | true
      | false
      | (expressão)
      | not fator

    """
    # ID E ID(EXPRESSOES)
    if self.ler_tipo() == 'identificador':
      self.verificar_declaracao(self.ler_palavra())
      self.proximo()
      if self.ler_simbolo() == '(':
        self.proximo()
        self.lista_de_expressoes()
        if self.ler_simbolo() == ')':
          self.proximo()
        else: self.erro( 'FATOR id(lista_de_expressao)- FALTANDO ")" ')
    
    elif self.ler_tipo() in ['integer','real','boolean']:
      self.proximo()
    
    # (EXPRESSÃO)
    elif self.ler_simbolo() == '(':
      self.proximo()
      self.expressao()
      if self.ler_simbolo() == ')':
        self.proximo()
      else: self.erro( 'FATOR (expressao) - FALTANDO ")" ')
    
    #NOT FATOR
    elif self.ler_simbolo() == 'not':
      self.proximo()
      self.fator()
    
    #BUGFIX
    elif self.ler_simbolo() == ')':
      pass
    
    else: self.erro("FALTANDO FATOR")

  @logar
  def sinal(self):
    """
    sinal →
      + | -
    """
    if self.ler_simbolo() in ['+','-']:
      self.proximo()

  @logar
  def op_relacional(self):
    """
    op_relacional →
      = | < | > | <= | >= | <>
    """
    if self.ler_tipo() == 'relacao':
      self.proximo()
  
  
  @logar
  def op_aditivo(self):
    """
    op_aditivo →
      + | - | or
    """
    if self.ler_tipo() == 'adicao':
      self.proximo()

  
  @logar
  def op_multiplicativo(self):
    """
    op_multiplicativo →
      * | / | and
    """
    if self.ler_tipo() == 'multiplicacao':
      self.proximo()

In [0]:

programa = """

Program Teste1;
Var       
    a, b, c : Integer;
    d, e, f : Real;
    g, h : Boolean;
		a : Integer; {causara erro, ja declarada no escopo}

Procedure Teste2(arg1,arg2 : Integer);
  Var
    a, b, c: Integer; {nao dara erro}
    arg1 : Real; {causara erro, ja declarada no escopo}
    x: Integer;

  Begin
  x := x + 1 * (3 + ( 2 * 4)) ;
  End;

Begin 
	x := x + 1 {dara erro, não declarada neste escopo}
End.



"""


resultado_lexico = Lexico(programa.lower()).start()
processado_sintatico = Sintatico(resultado_lexico, debug=False).start()

ERRO: Variavel a já declarada nesse escopo
ERRO: Variavel arg1 já declarada nesse escopo
ERRO: variavel x não declarada
ERRO: variavel x não declarada
Análise Sintática e Semantica Concluida.
